## Cargamos todas las bibliotecas necesarias para ejecutar todos los métodos del jupyter. 

Es buena costumbre cargarlos al principio, asi no se estar cargando en memoria repetidas veces cada paquete

In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_validate

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Creación de funciones auxiliares

In [2]:
def discretize(df, column):
    q1 = df_final[column].quantile(.25)
    q2 = df_final[column].quantile(.5)
    q3 = df_final[column].quantile(.75)
    df[column+"_in_q1"] = 0
    df[column+"_in_q2"] = 0
    df[column+"_in_q3"] = 0
    df[column+"_in_q4"] = 0
    df[column+"_in_q1"] = list(map(lambda x: 1 if x <=q1 else 0, df[column]))
    df[column+"_in_q2"] = list(map(lambda x: 1 if (q1 < x and x <=q2) else 0, df[column]))
    df[column+"_in_q3"] = list(map(lambda x: 1 if (q2 < x and x <=q3) else 0, df[column]))
    df[column+"_in_q4"] = list(map(lambda x: 1 if x>q3 else 0, df[column]))
    df.drop(column, axis=1, inplace=True)
    return df

In [3]:
def is_object(df,column,object_name):
    df["is_"+str(object_name)] = list(map(lambda x: 1 if x == object_name else 0, df[column]))
    return df
    

In [4]:
def find_and_replace_nans(df,col,rate_to_complete):
    
    index_list = df[col][df[col].isnull()].sample(frac=rate_to_complete,random_state=1).index.tolist()
    
    if len(index_list)>0:
        moda = df[(~df[col].isnull())][col].mode()[0]
        df.loc[index_list,col] = moda
        print("Completando columna " + str(col))
        print("Moda: " + str(moda))
        print("Cantidad restante de NULLs en columna " + str(col) + ": "+str(df[col][df[col].isnull()].shape))
        
    return df
    
def complete_nans(df,null_cols,rate_to_complete,method):
    if method=="moda_por_clase":
        for col in null_cols:
            print("Completando clase 0")
            df0 = find_and_replace_nans(df[df.target==0.0],col,rate_to_complete)
            print("Completando clase 1")
            df1 = find_and_replace_nans(df[df.target==1.0],col,rate_to_complete)
            df = pd.concat([df0,df1])
    else:
        for col in null_cols:
            df = find_and_replace_nans(df,col,rate_to_complete)
    
    return df.dropna()
            
            
        

In [5]:
def preprocess(df, metodo, rate_to_complete):
    
    # Extraemos las filas de la columna 'model' que tienen valores NaNs y luego los completamos con un valor por defecto
    
    #model_null_indexes = df[df.model.isnull()].index.tolist()
    #df.loc[model_null_indexes,"model"] = "other other"
    
    df['model'] = df['model'].fillna('other other')
    
    # Obtenemos las columnas que tienen al menos un NaN
    cols_in_null = df.columns[df.isnull().sum(axis=0)>0].tolist()
    
    # Los completamos con un rate_to_complete y utilizando un determinado metodo
    df_comp = complete_nans(df,cols_in_null,rate_to_complete,metodo)
    
    # Discretizamos las columnas que ya sabemos que tienen una distribucion dada (Chi, Normal)
    # (Se usan cuantiles para ver en qué segmento caen)
    for column in ["diff_time_days_minnone","diff_time_dayssearch engine hit","COUNT(users_logs)","number_visits_model","diff_time_daysbrand listing","diff_time_daysnone"]:
        df_comp = discretize(df_comp,column)
    
    # De la columna 'model' extraemos marca y modelo del celular visitado o  comprado
    df_comp["marca"] = df_comp.model.str.split(" ",1,expand=True)[0]
    df_comp["modelo"] = df_comp.model.str.split(" ",1,expand=True)[1]
    
    # Determinamos de que marca se trata
    df_comp["is_iphone"] = list(map(lambda x: 1 if x=="iPhone" else 0, df_comp["marca"]))
    df_comp["is_samsung"] = list(map(lambda x: 1 if x=="Samsung" else 0, df_comp["marca"]))
    df_comp["is_motorola"] = list(map(lambda x: 1 if x=="Motorola" else 0, df_comp["marca"]))
    df_comp["is_lg"] = list(map(lambda x: 1 if x=="LG" else 0, df_comp["marca"]))
    df_comp["is_sony"] = list(map(lambda x: 1 if x=="Sony" else 0, df_comp["marca"]))
    df_comp["is_other"] = list(map(lambda x: 1 if (x!="Sony" and x!="LG" and x!="Motorola" and x!="Samsung" and x!="iPhone") else 0, df_comp["marca"]))
    
    # determinamos de que modelo se trata
    pop_models = ['6','5s','6S','Galaxy J5','7','7 Plus','Galaxy S8','Galaxy S7','Galaxy S7 Edge','Galaxy J7 Prime','Moto G2 3G Dual','Galaxy S6 Edge','Galaxy S6 Flat','5c','6S Plus','Galaxy J7','6 Plus','Moto G4 Plus','SE','Galaxy Gran Prime Duos TV','4S','Galaxy S8 Plus','5','Moto G3 4G','Galaxy A5 2017','Moto X Play 4G Dual','Moto G5 Plus','Galaxy A7 2017','Moto X2','Galaxy A5']
    for model in pop_models:
        df_comp = is_object(df_comp,"modelo",model)

    df_comp["is_other_model"] = list(map(lambda x: 1 if x not in pop_models else 0, df_comp["modelo"]))
    
    # Eliminamos las columnas con strings
    df_comp.drop(["model","marca","modelo"],axis=1,inplace=True)
    
    return df_comp




## Cargamos los datasets para construir el dataset final

In [6]:
df_first = pd.read_csv('../../data_aa/first_part_dataset.csv')

df_second = pd.read_csv('../../data_aa/second_part_dataset.csv')

df_thrid = pd.read_csv('../../data_aa/thrid_part_dataset.csv')

In [7]:
df_first = df_first.rename(columns={'person_id': 'person'})

df_first = df_first.drop(columns=['Unnamed: 0'])

In [8]:
df_second.columns.values

array(['Unnamed: 0', 'person', 'diff_time_days_minnone',
       'diff_time_daysnone', 'diff_time_dayscheckout',
       'diff_time_daysconversion', 'diff_time_daysbrand listing',
       'diff_time_dayssearch engine hit', 'diff_time_start_end', 'model',
       'number_visits_model'], dtype=object)

In [9]:
df_second = df_second.drop(columns=['Unnamed: 0'])

In [10]:
df_thrid.columns.values

array(['Unnamed: 0', 'person', 'COUNT(users_logs)', 'avg_time_events'],
      dtype=object)

In [11]:
df_thrid = df_thrid.drop(columns=['Unnamed: 0'])

In [12]:
df_final = pd.merge(df_first, df_second, on='person', how='left')

df_final = pd.merge(df_final, df_thrid, on='person', how='left')

In [13]:
df_final.target.isin([1.0]).shape

(19414,)

Significado de las columnas:

 - Las columnas 'Android', 'BlackBerry', 'Chrome', 'FreeBSD', 'Linux', 'Mac', 'Tizen', 'Ubuntu', 'Unknown', 'Windows','iOS' indican la cantidad de veces que cada usuario ingreso a la pagina de Trocafone utilizando uno de esos esos sistemas operativos. Por ejemplo, si el valor de 'Android' para un usuario es 2 siginfica que ese usuario entro a la pagina dos veces desde un dispositivo con un sistema operativo Android
 
 - Las columnas 'Computer', 'Smartphone', 'Tablet' indican la cantidad de veces que cada usuario ingreso a la pagina de Trocafone utilizando uno de esos dispositivos
 
 - Las columnas 'ad campaign hit', 'brand listing', 'checkout', 'conversion', 'generic listing', 'lead','search engine hit', 'searched products', 'staticpage','viewed product', 'visited site' indican la cantidad de veces que cada usuario realizo cada uno de esos eventos
 
 - La columna 'diff_time_days_minnone' indica la cantidad de tiempo que paso desde el primer evento que realizo el usuario hasta el 2018/06/01
 
 - La columna 'diff_time_daysnone' indica la cantidad de tiempo que paso desde el ultimo evento que el usuario realizo en la pagina hasta el 2018/06/01
 
 - La columna 'diff_time_dayscheckout' indica la cantidad de tiempo que paso desde que el usuario realizo el ultimo checkout hasta el 2018/06/01
 
 - La columna 'diff_time_daysconversion' indica la cantidad de tiempo que paso desde que el usuario realizo la ultima conversion hasta el 2018/06/01
 
 - La columna 'diff_time_daysbrand listing' indica la cantidad de tiempo que paso desde que el usuario realizo la ultima brand listing hasta el 2018/06/01
 
 - La columna 'diff_time_dayssearch engine hit' indica la cantidad de tiempo que paso desde que el usuario realizo el ultimo engine hit hasta el 2018/06/01
 
 - La columna 'diff_time_start_end' indica la cantidad de tiempo que paso entre el primer evento que realizo el usuario en la pagina y el ultimo
 
 - La columna 'model' indica el modelo de celuar que el usuario mas veces visito
 
 - La columna 'number_visits_model' indica cuantas veces el usuario visito el celular que mas visito
 
 - La columna 'COUNT(users_logs)' indica la cantidad de eventos que el usuario realizo en la pagina
 
 - La columna 'avg_time_events' indica el tiempo promedio que paso entre los eventos realizados por cada usuario

### Eliminamos las columnas con muy pocas variables o alta corrleación (Ej. Windows ~ Computer)

In [14]:
df_final.drop(["BlackBerry","Chrome","FreeBSD","Linux","Mac","Other","Tizen","Ubuntu","Unknown","Windows","person"],
              axis=1,
              inplace=True)

### Transformamos algunas columnas para que su distribución se ajuste a una Chi cuadrado o Normal. Luego discretizamos por cuantiles

In [15]:
df_final.loc[:,"diff_time_daysnone"] = np.log10(df_final["diff_time_daysnone"]+1.)
df_final.loc[:,"diff_time_days_minnone"] = np.log10(df_final.diff_time_days_minnone+1.)
df_final.loc[:,"diff_time_dayssearch engine hit"] = np.log10(df_final["diff_time_dayssearch engine hit"]+1.0)
df_final.loc[:,"COUNT(users_logs)"] = np.log10(df_final["COUNT(users_logs)"]+1.)
df_final.loc[:,"number_visits_model"] = np.log10(1.+df_final.number_visits_model)
df_final.loc[:,"diff_time_daysbrand listing"] = np.log10(01.+df_final["diff_time_daysbrand listing"])

In [16]:
df_final['model'].head()

0                         NaN
1                    iPhone 7
2                   iPhone 6S
3    Motorola Moto G2 3G Dual
4      Samsung Galaxy S8 Plus
Name: model, dtype: object

### En el siguiente metodo  se realiza todo el preprocess de los datos.

In [17]:
df_completo = preprocess(df_final.copy(),"moda",1.)

Completando columna diff_time_dayscheckout
Moda: 2.2
Cantidad restante de NULLs en columna diff_time_dayscheckout: (0,)
Completando columna diff_time_daysconversion
Moda: 1.2
Cantidad restante de NULLs en columna diff_time_daysconversion: (0,)
Completando columna diff_time_daysbrand listing
Moda: 0.11394335230683679
Cantidad restante de NULLs en columna diff_time_daysbrand listing: (0,)
Completando columna diff_time_dayssearch engine hit
Moda: 0.11394335230683679
Cantidad restante de NULLs en columna diff_time_dayssearch engine hit: (0,)
Completando columna number_visits_model
Moda: 0.3010299956639812
Cantidad restante de NULLs en columna number_visits_model: (0,)
Completando columna avg_time_events
Moda: 0.0
Cantidad restante de NULLs en columna avg_time_events: (0,)


### Vamos a seleccionar solo 980 casos que no realizaron una compra para balancear el dataset

In [18]:
df_completo_reduced_1 = df_completo[df_completo['target']==1]
print(len(df_completo_reduced_1.index))

980


In [19]:
df_completo_reduced_0 = df_completo[df_completo['target']==0].sample(n=980, random_state=1)
print(len(df_completo_reduced_0.index))

980


In [20]:
df_train = pd.concat([df_completo_reduced_1, df_completo_reduced_0])
del df_completo_reduced_0, df_completo_reduced_1

### Nos quedamos con aquellas columnas que no sean target

In [21]:
feature_columns = list(set(df_train.columns.tolist())-set(["target"]))

### Obtenemos el dataset de test y lo eliminamos del dataset de train

In [22]:
df_test = df_train.sample(frac=0.2, random_state=2)
df_train.drop(df_test.index,axis=0,inplace=True)
train_indexes = df_train.index.tolist()

In [23]:
len(df_test)

392

### Verificamos el balance de categorías en cada datset

In [24]:
N_train = df_train.shape[0]*1.
N_test = df_test.shape[0]*1.

N_train_1 = df_train.target.sum()
N_train_0 = df_train.shape[0]*1.-N_train_1

N_test_1 = df_test.target.sum()
N_test_0 = df_test.shape[0]*1.-N_test_1

In [25]:
print("Porcentaje de casos positivos en el train: %" + str(100.0*N_train_1/N_train))
print("Porcentaje de casos positivos en el test: %" + str(100.0*N_test_1/N_test))

Porcentaje de casos positivos en el train: %50.255102040816325
Porcentaje de casos positivos en el test: %48.97959183673469


## Utilizando únicamente el dataset de train ejecutamos 5-fold Cross Validation

In [26]:
scoring = ['accuracy','roc_auc']

In [27]:
dtree = DecisionTreeClassifier(max_depth=3, criterion='gini') 

In [28]:
scores = cross_validate(dtree, # Instancia de árbol a entrenar en cada fold
                        df_train[feature_columns].values, # X_train features
                        df_train["target"].values, # Y_train targets
                        scoring=scoring, # Pedimos que para fold se ejecute esta lista de scorings
                        return_train_score = True, # Queremos ver los scorings de los training folds
                        return_estimator = True, # Ademas, pedimos que se nos devuelvan los dtrees de entrenados
                        cv=5, # Establecemos la cantidad de particiones para realizar la Validación cruzada
                        n_jobs = -1 # Pedimos que se ejecute en paralelo, utilizando todos los cores
                       )



### Cargamos los resultados en un DF de Pandas

In [29]:
cv_5_fold_resultados = pd.DataFrame(scores)
cv_5_fold_resultados.drop(["fit_time","score_time","estimator"],axis=1,inplace=True)

In [30]:
cv_5_fold_resultados

,test_accuracy,train_accuracy,test_roc_auc,train_roc_auc
0,0.732484,0.767145,0.791728,0.828612
1,0.767516,0.763955,0.836356,0.824302
2,0.707006,0.779904,0.753185,0.842019
3,0.760383,0.768127,0.802344,0.833754
4,0.769968,0.764143,0.814368,0.830726


### Calculamos los estadisticos de cada experimento, tanto para train como para test

In [31]:
cv_5_fold_resultados.describe()

,test_accuracy,train_accuracy,test_roc_auc,train_roc_auc
count,5.000000,5.000000,5.000000,5.000000
mean,0.747472,0.768655,0.799596,0.831883
std,0.027095,0.006549,0.030793,0.006629
min,0.707006,0.763955,0.753185,0.824302
25%,0.732484,0.764143,0.791728,0.828612
50%,0.760383,0.767145,0.802344,0.830726
75%,0.767516,0.768127,0.814368,0.833754
max,0.769968,0.779904,0.836356,0.842019


### Seleccionamos el árbol que mejor accuracy haya tenido durante el train

In [32]:
best_dtree_index = np.argmax(scores["test_accuracy"]*scores["test_roc_auc"])

In [33]:
best_btree = scores["estimator"][best_dtree_index]

### Evaluamos su accuracy sobre el 20% restante del dataset que no formó parte del 5-fold CV

In [34]:
y_esperado = df_test.target.values
y_pred = best_btree.predict(df_test[feature_columns].values)

In [35]:
print("Score validation: " + str(accuracy_score(y_esperado, y_pred)))

Score validation: 0.7831632653061225


In [36]:
# dot_data = StringIO()
# export_graphviz(best_btree, out_file=dot_data,  
#                 filled=True, rounded=True,
#                 special_characters=True)
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# Image(graph.create_png())

## Volvemos a correr el 5-fold CV pero completado el dataset de train con un dado porcentaje de completitud

In [37]:
df_train_80 = preprocess(df_final.loc[train_indexes].copy(),"moda",.25)

Completando columna diff_time_dayscheckout
Moda: 0.3
Cantidad restante de NULLs en columna diff_time_dayscheckout: (406,)
Completando columna diff_time_daysconversion
Moda: 1.0
Cantidad restante de NULLs en columna diff_time_daysconversion: (1003,)
Completando columna diff_time_daysbrand listing
Moda: 0.07918124604762482
Cantidad restante de NULLs en columna diff_time_daysbrand listing: (490,)
Completando columna diff_time_dayssearch engine hit
Moda: 0.07918124604762482
Cantidad restante de NULLs en columna diff_time_dayssearch engine hit: (287,)
Completando columna number_visits_model
Moda: 0.3010299956639812
Cantidad restante de NULLs en columna number_visits_model: (46,)
Completando columna avg_time_events
Moda: 0.0
Cantidad restante de NULLs en columna avg_time_events: (5,)


c:\users\bangho\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
c:\users\bangho\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\bangho\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

In [38]:
df_final.loc[train_indexes].shape

(1568, 28)

In [39]:
scores = cross_validate(dtree, # Instancia de árbol a entrenar en cada fold
                        df_train_80[feature_columns].values, # X_train features
                        df_train_80["target"].values, # Y_train targets
                        scoring=scoring, # Pedimos que para fold se ejecute esta lista de scorings
                        return_train_score = True, # Queremos ver los scorings de los training folds
                        return_estimator = True, # Ademas, pedimos que se nos devuelvan los dtrees de entrenados
                        cv=5, # Establecemos la cantidad de particiones para realizar la Validación cruzada
                        n_jobs = -1 # Pedimos que se ejecute en paralelo, utilizando todos los cores
                       )

In [40]:
cv_5_fold_resultados = pd.DataFrame(scores)
cv_5_fold_resultados.drop(["fit_time","score_time","estimator"],axis=1,inplace=True)

In [41]:
cv_5_fold_resultados

,test_accuracy,train_accuracy,test_roc_auc,train_roc_auc
0,0.682540,0.714286,0.655925,0.806379
1,0.612903,0.772358,0.615135,0.835405
2,0.639344,0.793522,0.675556,0.850739
3,0.688525,0.789474,0.730000,0.848535
4,0.622951,0.740891,0.676111,0.838295


In [42]:
cv_5_fold_resultados.describe()

,test_accuracy,train_accuracy,test_roc_auc,train_roc_auc
count,5.000000,5.000000,5.000000,5.000000
mean,0.649253,0.762106,0.670545,0.835871
std,0.034502,0.033830,0.041458,0.017729
min,0.612903,0.714286,0.615135,0.806379
25%,0.622951,0.740891,0.655925,0.835405
50%,0.639344,0.772358,0.675556,0.838295
75%,0.682540,0.789474,0.676111,0.848535
max,0.688525,0.793522,0.730000,0.850739


In [43]:
best_dtree_index = np.argmax(scores["test_accuracy"]*scores["test_roc_auc"])

In [44]:
best_btree = scores["estimator"][best_dtree_index]

In [45]:
y_esperado = df_test.target.values
y_pred = best_btree.predict(df_test[feature_columns].values)

In [46]:
print("Score validation: " + str(accuracy_score(y_esperado, y_pred)))

Score validation: 0.6045918367346939


In [47]:
# dot_data = StringIO()
# export_graphviz(best_btree, out_file=dot_data,  
#                 filled=True, rounded=True,
#                 special_characters=True)
# graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
# Image(graph.create_png())

## 2.4 Tolerancia al ruido

In [65]:
import random

#### Creamos una funcion que genera un array con valores 0 y numeros random

In [68]:
def create_array_noise(elements_in_column, percentage_noise):
    
    number_noise = round(elements_in_column*percentage_noise)
    
    mu, sigma = 0, 0.1
    
    noise = np.random.normal(mu, sigma, [number_noise,1])
    
    array_noise = noise.tolist()
    
    array_noise_final = []
    
    for item in array_noise:
        
        array_noise_final.append(item[0])
    
    array_no_noise = [0]*(elements_in_column - number_noise)
    
    noise = array_noise_final + array_no_noise
    
    random.shuffle(noise)
    
    return noise    

#### Creamos una funcion que, utilizando la funcion anterior, agrega un % de ruido al atributo que indiquemos de un dataset determinado

In [75]:
def generate_dataset_with_noise(dataframe, name_attribute, percentage_noise):
    
    number_observations = len(dataframe[name_attribute].values)
    
    array_noise = create_array_noise(number_observations, percentage_noise)
    
    dataframe[name_attribute] = dataframe[name_attribute] + array_noise
    
    return dataframe

In [ ]:
array_of_noise_percentages = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35]



In [ ]:
# b) Tenemos que usar todos los datos en df_train? Supongo que si

# c) No entendi el tema de la profundidad

# d) Tenemos que volver a entrar un modelo de arboles de decision y ver como performa?


In [78]:
generate_dataset_with_noise(df_train, 'visited site', 0.1)['visited site'].head()

13     4.000000
17     3.047724
78     0.993775
127    1.000000
254    7.000000
Name: visited site, dtype: float64

In [71]:
len(df_train.index)

1568

In [74]:
df_train['visited site'].head()

13     4.0
17     3.0
78     1.0
127    1.0
254    7.0
Name: visited site, dtype: float64

In [63]:
clean_signal['A'] = clean_signal['A'] + create_array_noise(2)
print(clean_signal)

          A    B
0  2.980688  2.0
1  4.877631  4.0


In [ ]:
# (DONE) Genera que solo un % de los valores del array de noise tengan otroa valores que 0

# (DONE) Elegir un atributo

# Tenemos que generar una familia de dataset con ruido

# Tenemos que correr el mejor modelo del punto dos para cada familia y obtener los resultados sobre el set de test

# Hacer un grafico con los resultados



## 3 Naive bayes

Obtenemos el dataset de "work", a partir del cual vamos a obtener el dataset de training y el de validacion

In [48]:
df_work = df_final_reduced.loc[~df_final_reduced.index.isin(df_test.index)]

print(len(df_work.index))

NameError: name 'df_final_reduced' is not defined

In [ ]:
df_work = df_work.sample(frac=1).reset_index(drop=True)

In [ ]:
print(len(df_work.index))

df_work.head(3)

### Sacamos columnas que no son numericas

In [ ]:
df_work = df_work.drop(columns=['person', 'model', 'marca', 'modelo'])

### Agregamos una columna al dataset que indica a que fold pertenece

In [ ]:
def generate_number_fold(dataframe, number_folds):
    
    number_rows = len(dataframe.index)
    
    rows_per_fold = round(number_rows/number_folds)
    
    array_number_folds = []
    
    index = 1
    
    while index < (number_folds + 1):
        
        if (number_folds - index) == 1:
            
            array_numbers = [index] * (number_rows - rows_per_fold*index)
        
            array_number_folds = array_number_folds + array_numbers
            
        else:
            
            array_numbers = [index] * rows_per_fold

            array_number_folds = array_number_folds + array_numbers
            
        index = index + 1

    dataframe['folds'] = array_number_folds
    
    return dataframe
        

### Filtramos por los folds que generamos con generate_number_fold() y vamos calculando las metricas que necesitamos tanto para training como para validation

In [ ]:
def applied_cross_validation_dt(dataframe, number_folds, depth, criterion):
    
    df_folds = generate_number_fold(dataframe, number_folds)
    
    index = 1
    
    array_accuracy_training = []
    
    array_accuracy_val = []
    
    array_auc_training = []
    
    array_auc_val = []
    
    while index < (number_folds + 1):
        
        df_specific_fold = df_folds[df_folds['folds']==index].drop(columns=['folds'])
        
        X = df_specific_fold.drop(columns=['target'])

        y = df_specific_fold['target']
        
        train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                    train_size=0.7,
                                                    test_size=0.3,
                                                    random_state=123)
        
        dtree=DecisionTreeClassifier(criterion=criterion, max_depth=depth)
        
        dtree.fit(train_X, train_y)
        
        predictions_val = dtree.predict(test_X)
        predictions_train = dtree.predict(train_X)
        
        proba_val = dtree.predict_proba(test_X)
        proba_train = dtree.predict_proba(train_X)
        
        acc_val = accuracy_score(test_y, predictions_val)
        acc_train = accuracy_score(train_y, predictions_train)
        
        auc_val = roc_auc_score(test_y, proba_val[:,1])
        auc_train = roc_auc_score(train_y, proba_train[:,1])
        
        array_accuracy_val.append(acc_val)
        
        array_accuracy_training.append(acc_train)
        
        array_auc_training.append(auc_val)
    
        array_auc_val.append(auc_train)
        
        index = index + 1
    
    return array_accuracy_training, array_accuracy_val, array_auc_training, array_auc_val

In [ ]:
applied_cross_validation_dt(df_work, 5, 3, 'gini')

In [ ]:
def generate_table(acc_train, acc_val, auc_train, auc_val, criterion, depth, number_folds):
    
    d = {'acc_train': acc_train, 'acc_val': acc_val, 'auc_train': auc_train, 'auc_val': auc_val}
    
    df = pd.DataFrame(data=d)
    
    df['criterion'] = [criterion] * number_folds
    
    df['max_depth'] = [depth] * number_folds
    
    return df

In [ ]:
acc_train_gini = applied_cross_validation_dt(df_work, 5, 3, 'gini')[0]

acc_val_gini = applied_cross_validation_dt(df_work, 5, 3, 'gini')[1]

auc_train_gini = applied_cross_validation_dt(df_work, 5, 3, 'gini')[2]

auc_val_gini = applied_cross_validation_dt(df_work, 5, 3, 'gini')[3]

df_gini_3 = generate_table(acc_train_gini, acc_val_gini, auc_train_gini, auc_val_gini, 'Gini', 3, 5)

df_gini_3 = df_gini_3.append(df_gini_3.mean(numeric_only=True), ignore_index=True)

df_gini_3

In [ ]:
def highlight_total_row(s):
    if s.B > 1.0:
        return ['background-color: yellow']*5
 
df.style.apply(highlight_greaterthan_1, axis=1)

In [ ]:
acc_train_gini = applied_cross_validation_dt(df_work, 5, 6, 'gini')[0]

acc_val_gini = applied_cross_validation_dt(df_work, 5, 6, 'gini')[1]

auc_train_gini = applied_cross_validation_dt(df_work, 5, 6, 'gini')[2]

auc_val_gini = applied_cross_validation_dt(df_work, 5, 6, 'gini')[3]

generate_table(acc_train_gini, acc_val_gini, auc_train_gini, auc_val_gini, 'Gini', 6, 5)